In [0]:
#Install tabula python wrapper
!pip install tabula-py

In [0]:
from tabula import read_pdf
import pandas as pd
from datetime import date

In [0]:
# Use the url from www.gob.mx
# TODO: check how their cms works, seems like every day they upload 4 pdfs... 
# check file/<number>/ and date at the end

positivos ="https://www.gob.mx/cms/uploads/attachment/file/543465/Tabla_casos_positivos_COVID-19_resultado_InDRE_2020.03.25.pdf"

In [0]:
# specify the area for page 1 since the column names are not being picked up by tabula
# A primer on PDF coordinate system to pass values to area param
# https://www.leadtools.com/help/leadtools/v19/dh/to/pdf-topics-pdfcoordinatesystem.html
df = read_pdf(positivos, area=[131, 0, 792, 792], stream=True, pages="1")

In [0]:
# Get the rest of the pages in a second data frame 
# TODO: seems like is not reading the first row, maybe because it doesnt have the upper border
df2 = read_pdf(positivos, stream=True , pages="all")

# Cleaning the first data frame

In [0]:
# Tabula outputs unformatted headers...
df[0].head()

In [0]:
# drop the first row with non usable data
df0 = df[0]

# Rename the columns
newcols = {
    'Unnamed: 0': 'No de caso', 
    'Unnamed: 1': 'Estado', 
    'Unnamed: 2': 'Sexo',
    'Unnamed: 3': 'Edad',
    'Unnamed: 4': 'Fecha de Inicio de síntomas',
    'real': 'Identificación de COVID-19 por RTPCR en tiempo real',
    'Unnamed: 5': 'Procedencia',
    'Unnamed: 6': 'Fecha de llegada a México'
}

# Throws a warning in colab, unfortunatly I'm still not proficient with python...
df0.rename(columns=newcols, inplace=True)

In [0]:
df0.head()

In [0]:
# Remove the extra columns:
columns = ['Unnamed: 8', 'Unnamed: 9']
df0 = df0.drop(labels=columns, axis=1)

In [0]:
df0.head()

## Concatenate with the other data frames

In [0]:
# Data frame 2 should have all pages...
len(df2)

In [33]:
## Remove page 1 since we already have it
df2 = df2[1:]
df2[0].head()

,47,NUEVO LEÓN,M,27,11/03/2020,confirmado,España,13/03/2020
0,48,NUEVO LEÓN,M,46,06/03/2020,confirmado,Estados Unidos,03/03/2020
1,49,GUERRERO,M,32,12/03/2020,confirmado,Contacto,NA
2,50,DURANGO,M,32,11/03/2020,confirmado,Alemania,11/03/2020
3,51,GUERRERO,F,22,09/03/2020,confirmado,España,12/03/2020
4,52,PUEBLA,F,27,05/03/2020,confirmado,España,09/03/2020


In [0]:
# Add column headers to data frames
cols = ['No de caso', 
    'Estado', 
    'Sexo',
    'Edad',
    'Fecha de Inicio de síntomas',
    'Identificación de COVID-19 por RTPCR en tiempo real',
    'Procedencia',
    'Fecha de llegada a México'
]

# Initialize list with the cleaned df 0 
dataFrames = [df0]

# Iterate through the dataframes in df2 and add the column headers
for dfx in df2:
    # Add header value as data
    first = dfx.columns
    first_row = dict( zip(cols, first))
    data = pd.DataFrame([first_row])
    dataframe = pd.concat([data, pd.DataFrame(dfx.values, columns=cols)])
    # append the dataframes
    dataFrames.append(dataframe)


In [0]:
# Concatenate the cleaned data frames
data = pd.concat(dataFrames,ignore_index=True)
data

## Export the data set:

In [0]:
today = date.today()
d1 = today.strftime("%Y-%m-%d")
filename = "confirmados-"+ d1+".csv"
data.to_csv(filename)